# RNN language model
Loosely based on [Zaremba et al. 2014](https://arxiv.org/abs/1409.2329), this example trains a word based RNN language model on Mikolov's PTB data with 10K vocab. It uses the `batchSizes` feature of `rnnforw` to process batches with different sized sentences. The `mb` minibatching function sorts sentences in a corpus by length and tries to group similarly sized sentences together. For an example that uses fixed length batches and goes across sentence boundaries see the [charlm](https://github.com/denizyuret/Knet.jl/blob/master/examples/charlm/charlm.ipynb) notebook.

In [1]:
using Knet
EPOCHS=10
RNNTYPE=:lstm
BATCHSIZE=64
EMBEDSIZE=128
HIDDENSIZE=256
VOCABSIZE=10000
NUMLAYERS=1
DROPOUT=0.5
LR=0.001
BETA_1=0.9
BETA_2=0.999
EPS=1e-08;

In [2]:
# Load data
include(Knet.dir("data","mikolovptb.jl"))
(trn,val,tst,vocab) = mikolovptb()
@assert VOCABSIZE == length(vocab)+1 # +1 for the EOS token
for x in (trn,val,tst,vocab); println(summary(x)); end

42068-element Array{Array{UInt16,1},1}
3370-element Array{Array{UInt16,1},1}
3761-element Array{Array{UInt16,1},1}
9999-element Array{String,1}


In [3]:
# Print a sample
println(tst[1])
println(vocab[tst[1]])

UInt16[0x008e, 0x004e, 0x0036, 0x00fb, 0x0938, 0x0195]
["no", "it", "was", "n't", "black", "monday"]


In [4]:
@doc mikolovptb

```
mikolovptb()
```

Read [PTB](https://catalog.ldc.upenn.edu/ldc99t42) text from Mikolov's [RNNLM](http://www.fit.vutbr.cz/~imikolov/rnnlm) toolkit which has been lowercased and reduced to a 10K vocabulary size.  Return a tuple (trn,dev,tst,vocab) where

```
trn::Vector{Vector{UInt16}}: 42068 sentences, 887521 words
dev::Vector{Vector{UInt16}}: 3370 sentences, 70390 words
tst::Vector{Vector{UInt16}}: 3761 sentences, 78669 words
vocab::Vector{String}: 9999 unique words
```


In [5]:
# Minibatch data into (x,y,b) triples. This is the most complicated part of the code:
# for language models x and y contain the same words shifted, x has an EOS in the beginning, y has an EOS at the end
# x,y = [ s11,s21,s31,...,s12,s22,...] i.e. all the first words followed by all the second words etc.
# b = [b1,b2,...,bT] i.e. how many sentences have first words, how many have second words etc.
# length(x)==length(y)==sum(b) and length(b)=length(s1)+1 (+1 because of EOS)
# sentences in batch should be sorted from longest to shortest, i.e. s1 is the longest sentence
function mb(sentences,batchsize)
    sentences = sort(sentences,by=length,rev=true)
    data = []; eos = VOCABSIZE
    for i = 1:batchsize:length(sentences)
        j = min(i+batchsize-1,length(sentences))
        sij = view(sentences,i:j)
        T = 1+length(sij[1])
        x = UInt16[]; y = UInt16[]; b = UInt16[]
        for t=1:T
            bt = 0
            for s in sij
                if t == 1
                    push!(x,eos)
                    push!(y,s[1])
                elseif t <= length(s)
                    push!(x,s[t-1])
                    push!(y,s[t])
                elseif t == 1+length(s)
                    push!(x,s[t-1])
                    push!(y,eos)
                else
                    break
                end
                bt += 1
            end
            push!(b,bt)
        end
        push!(data,(x,y,b))
    end
    return data
end

mbtrn = mb(trn,BATCHSIZE)
mbval = mb(val,BATCHSIZE)
mbtst = mb(tst,BATCHSIZE)
map(length,(mbtrn,mbval,mbtst))

(658, 53, 59)

In [6]:
# Define model
function initmodel()
    w(d...)=KnetArray(xavier(Float32,d...))
    b(d...)=KnetArray(zeros(Float32,d...))
    r,wr = rnninit(EMBEDSIZE,HIDDENSIZE,rnnType=RNNTYPE,numLayers=NUMLAYERS,dropout=DROPOUT)
    wx = w(EMBEDSIZE,VOCABSIZE)
    wy = w(VOCABSIZE,HIDDENSIZE)
    by = b(VOCABSIZE,1)
    return r,wr,wx,wy,by
end;

In [7]:
@doc rnninit

```
rnninit(inputSize, hiddenSize; opts...)
```

Return an `(r,w)` pair where `r` is a RNN struct and `w` is a single weight array that includes all matrices and biases for the RNN. Keyword arguments:

  * `rnnType=:lstm` Type of RNN: One of :relu, :tanh, :lstm, :gru.
  * `numLayers=1`: Number of RNN layers.
  * `bidirectional=false`: Create a bidirectional RNN if `true`.
  * `dropout=0.0`: Dropout probability. Ignored if `numLayers==1`.
  * `skipInput=false`: Do not multiply the input with a matrix if `true`.
  * `dataType=Float32`: Data type to use for weights.
  * `algo=0`: Algorithm to use, see CUDNN docs for details.
  * `seed=0`: Random number seed. Uses `time()` if 0.
  * `winit=xavier`: Weight initialization method for matrices.
  * `binit=zeros`: Weight initialization method for bias vectors.
  * `usegpu=(gpu()>=0)`: GPU used by default if one exists.

RNNs compute the output h[t] for a given iteration from the recurrent input h[t-1] and the previous layer input x[t] given matrices W, R and biases bW, bR from the following equations:

`:relu` and `:tanh`: Single gate RNN with activation function f:

```
h[t] = f(W * x[t] .+ R * h[t-1] .+ bW .+ bR)
```

`:gru`: Gated recurrent unit:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
r[t] = sigm(Wr * x[t] .+ Rr * h[t-1] .+ bWr .+ bRr) # reset gate
n[t] = tanh(Wn * x[t] .+ r[t] .* (Rn * h[t-1] .+ bRn) .+ bWn) # new gate
h[t] = (1 - i[t]) .* n[t] .+ i[t] .* h[t-1]
```

`:lstm`: Long short term memory unit with no peephole connections:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
f[t] = sigm(Wf * x[t] .+ Rf * h[t-1] .+ bWf .+ bRf) # forget gate
o[t] = sigm(Wo * x[t] .+ Ro * h[t-1] .+ bWo .+ bRo) # output gate
n[t] = tanh(Wn * x[t] .+ Rn * h[t-1] .+ bWn .+ bRn) # new gate
c[t] = f[t] .* c[t-1] .+ i[t] .* n[t]               # cell output
h[t] = o[t] .* tanh(c[t])
```


In [8]:
# Define loss and its gradient
function predict(ws,xs,bs;pdrop=0)
    r,wr,wx,wy,by = ws
    x = wx[:,xs] # xs=(ΣBt) x=(X,ΣBt)
    x = dropout(x,pdrop)
    (y,_) = rnnforw(r,wr,x,batchSizes=bs) # y=(H,ΣBt)
    y = dropout(y,pdrop)
    return wy * y .+ by  # return=(V,ΣBt)
end

loss(w,x,y,b;o...) = nll(predict(w,x,b;o...), y)

lossgradient = gradloss(loss);

In [9]:
@doc rnnforw

```
rnnforw(r, w, x[, hx, cx]; batchSizes, hy, cy)
```

Returns a tuple (y,hyout,cyout,rs) given rnn `r`, weights `w`, input `x` and optionally the initial hidden and cell states `hx` and `cx` (`cx` is only used in LSTMs).  `r` and `w` should come from a previous call to `rnninit`.  Both `hx` and `cx` are optional, they are treated as zero arrays if not provided.  The output `y` contains the hidden states of the final layer for each time step, `hyout` and `cyout` give the final hidden and cell states for all layers, `rs` is a buffer the RNN needs for its gradient calculation.

The boolean keyword arguments `hy` and `cy` control whether `hyout` and `cyout` will be output.  By default `hy = (hx!=nothing)` and `cy = (cx!=nothing && r.mode==2)`, i.e. a hidden state will be output if one is provided as input and for cell state we also require an LSTM.  If `hy`/`cy` is `false`, `hyout`/`cyout` will be `nothing`. `batchSizes` can be an integer array that specifies non-uniform batch sizes as explained below. By default `batchSizes=nothing` and the same batch size, `size(x,2)`, is used for all time steps.

The input and output dimensions are:

  * `x`: (X,[B,T])
  * `y`: (H/2H,[B,T])
  * `hx`,`cx`,`hyout`,`cyout`: (H,B,L/2L)
  * `batchSizes`: `nothing` or `Vector{Int}(T)`

where X is inputSize, H is hiddenSize, B is batchSize, T is seqLength, L is numLayers.  `x` can be 1, 2, or 3 dimensional.  If `batchSizes==nothing`, a 1-D `x` represents a single instance, a 2-D `x` represents a single minibatch, and a 3-D `x` represents a sequence of identically sized minibatches.  If `batchSizes` is an array of (non-increasing) integers, it gives us the batch size for each time step in the sequence, in which case `sum(batchSizes)` should equal `div(length(x),size(x,1))`. `y` has the same dimensionality as `x`, differing only in its first dimension, which is H if the RNN is unidirectional, 2H if bidirectional.  Hidden vectors `hx`, `cx`, `hyout`, `cyout` all have size (H,B1,L) for unidirectional RNNs, and (H,B1,2L) for bidirectional RNNs where B1 is the size of the first minibatch.


In [10]:
# Train and test loops
function train(model,data,optim)
    Σ,N=0,0
    for (x,y,b) in data
        grads,loss1 = lossgradient(model,x,y,b;pdrop=DROPOUT)
        update!(model, grads, optim)
        n = length(y)
        Σ,N = Σ+n*loss1, N+n
    end
    return Σ/N
end

function test(model,data)
    Σ,N=0,0
    for (x,y,b) in data
        loss1 = loss(model,x,y,b)
        n = length(y)
        Σ,N = Σ+n*loss1, N+n
    end
    return Σ/N
end;

In [12]:
#using JLD to save the model
model = optim = nothing; 
Knet.gc() # free gpu memory
if !isfile("rnnlm.jld")
    # Initialize and train model
    model = initmodel()
    optim = optimizers(model,Adam,lr=LR,beta1=BETA_1,beta2=BETA_2,eps=EPS)
    for epoch=1:EPOCHS
        @time global j1 = train(model,mbtrn,optim)  # ~39 seconds
        @time global j2 = test(model,mbval)         # ~1 second
        @time global j3 = test(model,mbtst)         # ~1 second
        println((epoch,exp(j1),exp(j2),exp(j3))); flush(stdout)  # prints perplexity = exp(negative_log_likelihood)
    end
    #save("rnnlm.jld","model",model)
else
    model = load("rnnlm.jld","model")
    @time global j1 = test(model,mbtrn)
    @time global j2 = test(model,mbval)
    @time global j3 = test(model,mbtst)
    println((EPOCHS,exp(j1),exp(j2),exp(j3))); flush(stdout)  # prints perplexity = exp(negative_log_likelihood)
end
summary(model)

 63.418385 seconds (1.34 M allocations: 539.577 MiB, 19.29% gc time)
  3.038773 seconds (49.12 k allocations: 3.443 MiB, 17.17% gc time)
  1.796301 seconds (37.88 k allocations: 3.529 MiB, 14.45% gc time)
(1, 740.4586f0, 1.2956388f6, 1.2024276f6)
 63.598776 seconds (1.35 M allocations: 539.792 MiB, 17.97% gc time)
  3.047761 seconds (49.12 k allocations: 3.443 MiB, 17.37% gc time)
  1.800283 seconds (37.88 k allocations: 3.529 MiB, 14.47% gc time)
(2, 578.68585f0, 55071.39f0, 51691.68f0)
 63.619360 seconds (1.35 M allocations: 539.792 MiB, 18.00% gc time)
  3.038184 seconds (49.12 k allocations: 3.443 MiB, 17.15% gc time)
  1.803893 seconds (37.88 k allocations: 3.529 MiB, 14.53% gc time)
(3, 420.61963f0, 32705.42f0, 31171.264f0)
 63.832536 seconds (1.35 M allocations: 539.792 MiB, 18.23% gc time)
  3.040227 seconds (49.12 k allocations: 3.443 MiB, 17.22% gc time)
  1.799548 seconds (37.88 k allocations: 3.529 MiB, 14.46% gc time)
(4, 354.73358f0, 9335.126f0, 8942.5205f0)
 63.635407 se

"Tuple{Knet.RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}"